In [1]:
FTRAIN = 'data/training.csv' #getting training data

Загрузка данных: изображений с лицами с меткой 1

In [2]:
import csv

def parse_csv(mark):
    csvfile = open(FTRAIN, 'r')
    reader = csv.reader(csvfile, delimiter='\n')
    my_list = list(reader)
    del my_list[:1]
    array = []
    for entry in my_list:
        entry = ''.join(''.join(entry).partition(' ')[1:])
        entry = entry.split()
        entry.append(mark) #marking images with faces
        array.append(entry)
        
    return array    
    
array = parse_csv("1")

In [3]:
import csv
from random import shuffle

def parse_csv_shuffle(mark):
    csvfile = open(FTRAIN, 'r')
    reader = csv.reader(csvfile, delimiter='\n')
    my_list = list(reader)
    del my_list[:1]
    array = []
    for entry in my_list:
        entry = ''.join(''.join(entry).partition(' ')[1:])
        entry = entry.split()
        shuffle(entry)
        entry.append(mark) #marking images with faces
        array.append(entry)
        
    return array    
    
array_shuffled = parse_csv_shuffle("0")

Подготовка данных

In [3]:
x_train = []
y_train = []

for entry in array:
    y_train.append(entry[len(entry) - 1])
    x_train.append(entry[0:len(entry) - 1])
    
#for entry in array_shuffled:
#    y_train.append(entry[len(entry) - 1])
#    x_train.append(entry[0:len(entry) - 1])

In [ ]:
import numpy

x_train = numpy.array(x_train)
x_train = x_train.reshape(len(x_train), len(x_train[1]))
x_train = x_train.astype('float32')
x_train /= 225

In [4]:
from keras.utils import np_utils 

y_train = np_utils.to_categorical(y_train, 2) 

Using Theano backend.


Создание модели

In [5]:
height, width, deph = 96,96,1
batch_size = 128 
num_epochs = 1 
hidden_size = 512 
num_classes = 2 

In [10]:
from keras.layers import Input, Dense 
from keras.models import Model  

def create_model():
    inp = Input(shape=(9215,))
    hidden_1 = Dense(hidden_size, activation ='relu')(inp)
    hidden_2 = Dense(hidden_size, activation = 'relu')(hidden_1) 
    out = Dense(num_classes, activation='softmax')(hidden_2) 
    model = Model(input = inp, output = out)
    return model
    
model = create_model()

In [7]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'] )

Обучение модели

In [8]:
model.fit(x_train, y_train, batch_size = batch_size, nb_epoch = num_epochs)

Epoch 1/1
7049/7049 [==============================] - 17s - loss: 0.0206 - acc: 0.9984     

In [ ]:
model.evaluate(X, y,verbose=1) #оценка модели на тестовом наборе 